# Creating a database with Postgres to store wealth data

### Functions to manage database

In [17]:
import psycopg2
import pandas as pd

def create_database():
    # Connect to default database
    try:
        conn = psycopg2.connect('host=127.0.0.1 dbname=postgres user=postgres password=root')
    except psycopg2.Error as e:
        print('Error connecting to database')
        print(e)
    
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    # Create new database
    try:
        cur.execute('DROP DATABASE IF EXISTS wealth_database')
        cur.execute('CREATE DATABASE wealth_database')
    except psycopg2.Error as e:
        print('Error creating database')
        print(e)

    # Close connection to default database
    conn.close()

    # Connect to database
    conn = psycopg2.connect('host=127.0.0.1 dbname=wealth_database user=postgres password=root')
    cur = conn.cursor()

    return cur, conn

def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

def drop_tabels(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()


In [67]:
cur, conn = create_database()

### Cleaning data

In [66]:
acc_data = pd.read_csv('data/Wealth-AccountData.csv')

# Cleaning data
acc_data_clean = acc_data[['Country Code', 'Series Code', '1995 [YR1995]','2000 [YR2000]', '2005 [YR2005]', '2010 [YR2010]', '2014 [YR2014]']]
new_columns_data = {
    'Country Code': 'country_code'
    , 'Series Code': 'series_code'
    , '1995 [YR1995]': 'year_1995'
    , '2000 [YR2000]': 'year_2000'
    , '2005 [YR2005]': 'year_2005'
    , '2010 [YR2010]': 'year_2010'
    , '2014 [YR2014]': 'year_2014'
}

acc_data_clean.rename(columns=new_columns_data,inplace=True)
acc_data_clean.dropna(inplace=True)

acc_data_clean.replace({
    'year_1995':{
        '..':0
    }
    , 'year_2000':{
        '..':0
    }
}, inplace=True)

acc_data_clean.head()

/tmp/ipykernel_49719/302370702.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_data_clean.rename(columns=new_columns_data,inplace=True)
/tmp/ipykernel_49719/302370702.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_data_clean.dropna(inplace=True)
/tmp/ipykernel_49719/302370702.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_data_clean.replace({


,country_code,series_code,year_1995,year_2000,year_2005,year_2010,year_2014
0,ALB,NW.HCA.TO,4.49E+10,4.36E+10,5.440000e+10,6.810000e+10,7.250000e+10
1,ALB,NW.HCA.PC,14072.79,14118.13,1.807230e+04,2.336286e+04,2.509002e+04
2,ALB,NW.HCA.FEMP.PC,3468.858,3424.007,2.222115e+03,2.889150e+03,2.957420e+03
3,ALB,NW.HCA.MEMP.PC,6507.931,6872.019,1.168935e+04,1.578992e+04,1.699138e+04
4,ALB,NW.HCA.FEMA.PC,4893.006,4695.071,2.886736e+03,3.613606e+03,3.719608e+03


In [41]:
acc_country = pd.read_csv('data/Wealth-AccountsCountry.csv')

# Cleaning data
acc_country_clean = acc_country[['Code', 'Short Name', 'Long Name', 'Region', 'Currency Unit']]
new_columns_country = {
    'Code': 'code'
    , 'Short Name': 'short_name'
    , 'Long Name': 'long_name'
    , 'Region': 'region'
    , 'Currency Unit': 'currency_unit'
}
acc_country_clean.rename(columns=new_columns_country, inplace=True)
acc_country_clean.dropna(inplace=True)
acc_country_clean

/tmp/ipykernel_49719/3215100340.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_country_clean.rename(columns=new_columns_country, inplace=True)
/tmp/ipykernel_49719/3215100340.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_country_clean.dropna(inplace=True)


,code,short_name,long_name,region,currency_unit
0,ALB,Albania,Republic of Albania,Europe & Central Asia,Albanian lek
1,ARG,Argentina,Argentine Republic,Latin America & Caribbean,Argentine peso
2,ARM,Armenia,Republic of Armenia,Europe & Central Asia,Armenian dram
3,AUS,Australia,Commonwealth of Australia,East Asia & Pacific,Australian dollar
4,AUT,Austria,Republic of Austria,Europe & Central Asia,Euro
...,...,...,...,...,...
141,VNM,Vietnam,Socialist Republic of Vietnam,East Asia & Pacific,Vietnamese dong
142,PSE,West Bank and Gaza,West Bank and Gaza,Middle East & North Africa,Israeli new shekel
143,YEM,Yemen,Republic of Yemen,Middle East & North Africa,Yemeni rial
144,ZMB,Zambia,Republic of Zambia,Sub-Saharan Africa,New Zambian kwacha


In [52]:
acc_series = pd.read_csv('data/Wealth-AccountSeries.csv')

# Cleaning data
acc_series_clean = acc_series[['Code', 'Indicator Name', 'Topic', 'Periodicity', 'Unit of measure', 'Reference period']]
new_columns_series = {
    'Code': 'code'
    , 'Indicator Name': 'indicator_name'
    , 'Topic': 'topic'
    , 'Periodicity': 'periodicity'
    , 'Unit of measure': 'unit_of_measure'
    , 'Reference period': 'reference_period'
}
acc_series_clean.rename(columns=new_columns_series, inplace= True)
acc_series_clean.head()

/tmp/ipykernel_49719/3119501796.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_series_clean.rename(columns=new_columns_series, inplace= True)


,code,indicator_name,topic,periodicity,unit_of_measure,reference_period
0,NW.HCA.TO,Human capital (constant 2018 US$),Human capital,Annual,Constant 2018 US$,1995-2018
1,NW.HCA.PC,Human capital per capita (constant 2018 US$),Human capital,Annual,Constant 2018 US$,1995-2018
2,NW.HCA.FEMP.PC,"Human capital per capita, employed female (con...",Human capital,Annual,Constant 2018 US$,1995-2018
3,NW.HCA.MEMP.PC,"Human capital per capita, employed male (const...",Human capital,Annual,Constant 2018 US$,1995-2018
4,NW.HCA.FEMA.PC,"Human capital per capita, female (constant 201...",Human capital,Annual,Constant 2018 US$,1995-2018


### Creating tables at the database

In [68]:
# Creating table account_data
data_table_create = (
    '''
    CREATE TABLE IF NOT EXISTS account_data(
        country_code VARCHAR
        , series_code VARCHAR
        , year_1995 numeric
        , year_2000 numeric
        , year_2005 numeric
        , year_2010 numeric
        , year_2014 numeric
    )
    '''
)

data_table_insert = (
    '''
    INSERT INTO account_data(
        country_code
        , series_code
        , year_1995
        , year_2000
        , year_2005
        , year_2010
        , year_2014
    )
    VALUES(%s, %s, %s, %s, %s, %s, %s)
    '''
)

cur.execute(data_table_create)
conn.commit()

for i, row in acc_data_clean.iterrows():
    cur.execute(data_table_insert, list(row))

conn.commit()

In [44]:
country_table_create = (
    '''
    CREATE TABLE IF NOT EXISTS account_country(
        code VARCHAR PRIMARY KEY
        , short_name VARCHAR
        , long_name VARCHAR
        , region VARCHAR
        , currency_unit VARCHAR
    )
    '''
)

country_table_insert = (
    '''
    INSERT INTO account_country(
        code
        , short_name
        , long_name
        , region
        , currency_unit
    )
    VALUES(%s,%s,%s,%s,%s)
    '''
)

cur.execute(country_table_create)
conn.commit()

for i, row in acc_country_clean.iterrows():
    cur.execute(country_table_insert, list(row))

conn.commit()

In [29]:
series_table_create = (
    '''
    CREATE TABLE IF NOT EXISTS account_serie(
        code VARCHAR PRIMARY KEY
        , indicator_name VARCHAR
        , topic VARCHAR
        , periodicity VARCHAR
        , unit_of_measure VARCHAR
        , reference_period VARCHAR
    )
    '''
)

series_table_insert = (
    '''
    INSERT INTO account_serie(
        code
        , indicator_name
        , topic
        , periodicity
        , unit_of_measure
        , reference_period
    )
    VALUES(%s,%s,%s,%s,%s,%s)
    '''
)

cur.execute(series_table_create)
conn.commit()

for i, row in acc_series_clean.iterrows():
    cur.execute(series_table_insert, list(row))

conn.commit()

In [69]:
conn.close()